In [1]:
from gtts import gTTS
import playsound
import pymysql
import os

In [2]:
# localhost 개인 db접속
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '0000',
    database = 'sound_db',
    charset = 'utf8'
)

cur = conn.cursor()

In [3]:
def get_docker_data(di, ob):
    case = f'{di} {ob} 있습니다.'
    query = 'select docker_data from sound where txt=\"{}\";'.format(case)
    cur.execute(query)
    data = cur.fetchall()
    conn.commit()

    if data:
        docker_data = data[0][0]
        return docker_data, case
    else:
        return None, case

In [4]:
def get_dieg_obeg(result_case):
  if result_case[0] == "왼쪽":
    dieg = "left"
  elif result_case[0] == "오른쪽":
    dieg = "right"

  if result_case[1] == "사람":
    obeg = "person"
  elif result_case[1] == "자전거":
    obeg = "bicycle"
  elif result_case[1] == "자동차":
    obeg = "car"
  elif result_case[1] == "오토바이":
    obeg = "motorcycle"
  elif result_case[1] == "버스":
    obeg = "bus"
  elif result_case[1] == "트럭":
    obeg = "truck"

  return dieg, obeg

In [5]:
def docker_download_file(docker_data, download_path, case):
    if os.path.exists(download_path):
        return
    
    # 데이터가 없으면
    if not docker_data:
        txt_path = case
        tts = gTTS(text=txt_path, lang='ko')
        tts.save(download_path)
        

    # 데이터가 있으면
    else:
        # 도커이미지상의 경로 지정
        !docker run -it --rm test:test sh -c "find /BlackboxSound -type f -name {case} -exec cp {} /tmp ; && tar -cf - /tmp | cat" > {download_path}

In [6]:
def download_file(dieg, obeg, docker_data, case):
    download_path=f"../BlackboxSound/{dieg}_{obeg}.mp3"
    docker_download_file(docker_data, download_path, case)
    playsound.playsound(download_path)

In [7]:
def play(result_case):
    di = result_case[0]
    ob = result_case[1]
    dieg, obeg = get_dieg_obeg(result_case)
    docker_data, case = get_docker_data(di, ob)
    download_file(dieg, obeg, docker_data, case)

In [8]:
result_case = [['오른쪽', '사람'],
                ['오른쪽', '사람'],
                ['오른쪽', '오토바이'],
                ['오른쪽', '사람'],
                ['왼쪽', '사람'],
                ['왼쪽', '자전거'],
                ['오른쪽', '오토바이']]

for case in result_case:
    play(case)